<a href="https://colab.research.google.com/github/czterdziesci4/polish-vehical-database-cepik/blob/main/Cepik_by_year_colab_executable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### cepik api

Dokumnetacja APi Cepik: https://api.cepik.gov.pl/doc
Filtrowanie: filter[<nazwa atrybutu>]=<szukana wartość>
    
    filter[<rodzaj-paliwa>]=<ENERGIA ELEKTRYCZNA>

In [1]:
import pandas as pd
import numpy as np
import json
import time
import requests
import os
from datetime import datetime
from google.colab import files


### Dictnionaries

#### Wojeództwa

In [2]:
# just use whatever you want - this is total avaiable list of wojewodztwo and years 
#wojew = ["02","04","06","08","10","12","14","16","18","20","22","24","26","28","30","32","XX"]
#years = ["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021","2022"]

wojew = ["02","04"]
years = ["2010","2011"]


# Regions codes
# code	województwo
# 02	DOLNOŚLĄSKIE
# 04	KUJAWSKO-POMORSKIE
# 06	LUBELSKIE
# 08	LUBUSKIE
# 10	ŁÓDZKIE
# 12	MAŁOPOLSKIE
# 14	MAZOWIECKIE
# 16	OPOLSKIE
# 18	PODKARPACKIE
# 20	PODLASKIE
# 22	POMORSKIE
# 24	ŚLĄSKIE
# 26	ŚWIĘTOKRZYSKIE
# 28	WARMIŃSKO-MAZURSKIE
# 30	WIELKOPOLSKIE
# 32	ZACHODNIOPOMORSKIE
# XX	NIEOKREŚLONE


In [3]:
#Based url

based_url = "https://api.cepik.gov.pl/pojazdy?&limit=500" 


# Filters
#filtr = '&filter[rodzaj-paliwa]=ENERGIA+ELEKTRYCZNA' # just sample filter
filtr = '&filter[rodzaj-paliwa]=ENERGIA+ELEKTRYCZNA&tylko-zarejestrowane=false&pokaz-wszystkie-pola=true'


# local machine - uncomment below baths if you use it at local machine - serch the code for to uncomment all: local machine
#upload_path = "J:/Dokumenty/Tomek/YT/samochody/cepik/grab/" # it was used on local machine 
#current_dic = "2022-05-15/" # it was used on local machine

base_name_xls = "Cepik_" 
# url


In [4]:
try:
    del cars
    del df
except Exception:
    print("no df to del")

no df to del


Main Window

In [11]:
# Wojewodztwo loop

for w in wojew:

    print ("-"*40)
    print("wojedzództwo:", w)
    
    wojewodz = "&wojewodztwo=" + w
    
    # Year loop
    print ("First year to dig:", years[0] )
    print ("Last year to dig:", years[-1] )
    
    for y in years:
        
        # on local machine
        # Make a directory 
        #path = os.path.join(upload_path, w , y) 
        #print ("y=",y)
        
        # local machine - uncomment below baths if you use it at local machine
        #path_to_save = upload_path + "/" + w + "/" + y + "/" 
        
        # was used on local machine
        # local machine 
        
        #try: 
        #    os.makedirs (path_to_save)
        #    print (path_to_save, " successfully created")
        #except Exception:
        #    print("dic: ", path_to_save, "aready exit" )
        #print ("path_to_save", path_to_save)
        
        print ("-"*20)
        print("Year:", y)
        
        period = "&data-od=" + y + "0101&data-do=" + y + "1231"
        
                
        # remove reco_no        
        try:
            del rec_no
        except Exception:
            print("no data to del")
   
        
        # seting up variable before the loop
        rec_no = 0
        step = 1
        finished = False 
        number_of_waits = 0
        
        while not finished:
            
            page = "&page=" + str(step)
            url = based_url + period +  filtr + wojewodz + page

            print ("We dig in:", url)
           

            #-------------collecting data -------    
            data_temp = requests.get(url) 
            #------------------------------------
            
            if data_temp.status_code != 200:
                number_of_waits = number_of_waits + 1
                print ("digging status NOK: ", data_temp.status_code, "wait for 5 sek.", number_of_waits , "times")
                time.sleep(5) # czekam 5 sek
            else:
                print ("digging OK -", "data statatus code:",data_temp.status_code)

                # normalizing json files                
                data = data_temp.json()
                df = pd.json_normalize(data, record_path =['data'])
                
                # how many times we need to iterate data to download
                rec_no = pd.json_normalize(data)
                rec_no = rec_no["meta.count"][0]
                records_tc =  rec_no
                rec_no = round(rec_no/500+1,0)

                print ("page/step: ",step," Records in woj: ",w, "records to download:",records_tc ," pages do download: ", rec_no)
                
                
                # ----------- aggregtion data into the year and woj ----------------
                # if first step lets make cars as a df
                if step == 1:
                    cars_in_years_and_woj = df
                else:
                    # add another set of data form steps in the common df
                    cars_in_years_and_woj = cars_in_years_and_woj.append(df)
                
                # ----------- aggregtion data into the year and woj ----------------
                
                
                # save each set of data from particular step steps into the csv
                now_file = "generated_" + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
                
                
                
                name_xls = "Cepik_" + y + "_woj_" +w + "_step_" + str(step) + "-" + now_file + ".csv"
                # df.to_csv(path_or_buf = name_xls)
                
        
                
                if step == rec_no:
                    finished = True
                    
                    # save one year of woj - all steps - into the csv
                    #print (os.path.join(upload_path, w , y) + base_name_xls + y + "_woj_" +w + "_total_" + "-" + now_file + ".csv")
                    
                    # local machine - uncomment below baths if you use it at local machine
                    #name_xls_total = path_to_save + base_name_xls + y + "_woj_" +w + "_total_" + "-" + now_file + ".csv"
                    name_xls_total = "Cepik" + y + "_woj_" +w + "_total_" + "-" + now_file + ".csv"
                    
                    # This is creationg a partial file (for verification of data)
                    # Data will be later appended but you may still download the partial files
                    # just uncomment below two lines 
                    #-----------------------
                    #cars_in_years_and_woj.to_csv(path_or_buf= name_xls_total)
                    #files.download(name_xls_total)
                    #-----------------------
                    
                    # Agregation of the WOJ - all years in WOJ
                    # Agregation data - all years in one Woj
                    #print ("y=",y,"years[0]=",years[0])
                    if y == years[0]:
                        all_cars_in_years = cars_in_years_and_woj
                        #print ("!!!!!! first stp i all_cars_in_years = cars_in_years_and_woj")
                    else:
                        # add another set of data form steps in the common df
                        all_cars_in_years = all_cars_in_years.append(cars_in_years_and_woj)

                    
                else:    
                    step = step + 1
               
                print ("-"*5)
    
    
    
    name_xls_all_years = "Cepik_WOJ-" + w + "_years_" + years[0] + "-" + years[-1] + "_total_" + "-" + now_file + ".csv"
    print ("saving woj:",w," to the path: ", name_xls_all_years)
    
    all_cars_in_years.to_csv(path_or_buf= name_xls_all_years)
    files.download(name_xls_all_years)
    del cars_in_years_and_woj
    
    # end of WOj loop

----------------------------------------
wojedzództwo: 02
First year to dig: 2010
Last year to dig: 2011
--------------------
Year: 2010
We dig in: https://api.cepik.gov.pl/pojazdy?&limit=500&data-od=20100101&data-do=20101231&filter[rodzaj-paliwa]=ENERGIA+ELEKTRYCZNA&tylko-zarejestrowane=false&pokaz-wszystkie-pola=true&wojewodztwo=02&page=1
digging OK - data statatus code: 200
page/step:  1  Records in woj:  02 records to download: 16  pages do download:  1.0
-----
--------------------
Year: 2011
We dig in: https://api.cepik.gov.pl/pojazdy?&limit=500&data-od=20110101&data-do=20111231&filter[rodzaj-paliwa]=ENERGIA+ELEKTRYCZNA&tylko-zarejestrowane=false&pokaz-wszystkie-pola=true&wojewodztwo=02&page=1
digging OK - data statatus code: 200
page/step:  1  Records in woj:  02 records to download: 14  pages do download:  1.0
-----
saving woj: 02  to the path:  Cepik_WOJ-02_years_2010-2011_total_-generated_2022_11_23_18_56_26.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

----------------------------------------
wojedzództwo: 04
First year to dig: 2010
Last year to dig: 2011
--------------------
Year: 2010
We dig in: https://api.cepik.gov.pl/pojazdy?&limit=500&data-od=20100101&data-do=20101231&filter[rodzaj-paliwa]=ENERGIA+ELEKTRYCZNA&tylko-zarejestrowane=false&pokaz-wszystkie-pola=true&wojewodztwo=04&page=1
digging OK - data statatus code: 200
page/step:  1  Records in woj:  04 records to download: 0  pages do download:  1.0
-----
--------------------
Year: 2011
We dig in: https://api.cepik.gov.pl/pojazdy?&limit=500&data-od=20110101&data-do=20111231&filter[rodzaj-paliwa]=ENERGIA+ELEKTRYCZNA&tylko-zarejestrowane=false&pokaz-wszystkie-pola=true&wojewodztwo=04&page=1
digging OK - data statatus code: 200
page/step:  1  Records in woj:  04 records to download: 3  pages do download:  1.0
-----
saving woj: 04  to the path:  Cepik_WOJ-04_years_2010-2011_total_-generated_2022_11_23_18_56_30.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Marging Files